In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Fill in the token and folder fields</h3>
# ================================================================ #

import os, time, urllib.request, json, string, psutil, uuid; from IPython.display import clear_output
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output
Ngrok_Token = "" #@param {type:"string"}
RPC_token = "" #@param {type:"string"}
Remote_folder = "" #@param {type:"string"}
Remote_token = "" #@param {type:"string"}

os.environ["RCLONE_CONFIG_MYDRIVE_TYPE"] = "drive"
os.environ["RCLONE_CONFIG_MYDRIVE_SCOPE"] = "drive"
os.environ["RCLONE_CONFIG_MYDRIVE_ROOT_FOLDER_ID"] = Remote_folder
os.environ["RCLONE_CONFIG_MYDRIVE_TOKEN"] = Remote_token

SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def CheckProcess(processName):
    for proc in psutil.process_iter():
        try:
            if processName.lower() in proc.name().lower():
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False;
  
def Start_Server():
  if CheckProcess("screen") == False or CheckProcess("aria2c main") == False:
    %cd /content/aria2_home
    !/usr/bin/screen -d -m -fa -S aria2c /usr/bin/aria2c --conf-path="/content/aria2_home/aria2.conf" --rpc-secret={RPC_token} --daemon=false

def init_ngrok():
  if os.path.isfile("/usr/local/bin/ngrok") == False:
      !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
      !unzip -qq -n ngrok-stable-linux-amd64.zip
      !mv ngrok /usr/local/bin/ngrok
      !rm -f /content/ngrok-stable-linux-amd64.zip
  else:
      !pkill ngrok

  get_ipython().system_raw('/usr/local/bin/ngrok authtoken $Ngrok_Token && /usr/local/bin/ngrok http 6800 &')
    
def get_url():
  time.sleep(10)
  with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
    data = json.loads(response.read().decode())
    (url) = data['tunnels'][0]['public_url']
    #print(f'{url}')
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">aria2box</h2><h4 style=\"font-family:Trebuchet MS;color:#446785;\">" \
                "HOST: "+ url.strip('https://') + "<br>PORT: 443<br>TOKEN: "+RPC_token+"</h4><h2>https://token:"+RPC_token+"@"+ url.strip('https://') +":443/jsonrpc</h2><br>"))

try:
  if os.path.isfile("/usr/bin/aria2c") == False:
    !rm -rf /content/sample_data/
    !apt update -qq -o=Dpkg::Use-Pty=0 > /dev/null
    !apt install -qq -o=Dpkg::Use-Pty=0 screen autossh aria2 > /dev/null
    !wget https://github.com/rclone/rclone/releases/download/v1.49.3/rclone-v1.49.3-linux-amd64.deb && dpkg -i rclone-v1.49.3-linux-amd64.deb && rm rclone-v1.49.3-linux-amd64.deb
    !git clone https://gitlab.com/mega036/aria2cloud-colab.git /content/aria2_home
    !chmod +x /content/aria2_home/.aria2/*.sh
  clear_output()
  Start_Server()
  init_ngrok()
  get_url()
  display(SuccessRun)
except:
  clear_output()
  display(UnsuccessfullyRun)

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>Test rclone setup</h3>
# ================================================================ #

import os
from google.colab import output

Remote_folder = "" #@param {type:"string"}
Remote_token = "" #@param {type:"string"}

os.environ["RCLONE_CONFIG_MYDRIVE_TYPE"] = "drive"
os.environ["RCLONE_CONFIG_MYDRIVE_SCOPE"] = "drive"
os.environ["RCLONE_CONFIG_MYDRIVE_ROOT_FOLDER_ID"] = Remote_folder
os.environ["RCLONE_CONFIG_MYDRIVE_TOKEN"] = Remote_token
if os.path.isfile("/usr/bin/rclone") == False:
  !wget https://github.com/rclone/rclone/releases/download/v1.49.3/rclone-v1.49.3-linux-amd64.deb && dpkg -i rclone-v1.49.3-linux-amd64.deb && rm rclone-v1.49.3-linux-amd64.deb
!rclone lsd mydrive:
